In [1]:
project_name="cnnclf"

In [2]:
import os

In [3]:
%pwd

'f:\\ML_Ops_learning\\Kidney_Disease_Classification_MLflow\\notebooks'

In [4]:
os.chdir("../")

In [5]:
%pwd

'f:\\ML_Ops_learning\\Kidney_Disease_Classification_MLflow'

MLFLOW_TRACKING_URI=https://dagshub.com/SunilKumar-ugra/Kidney_Disease_Classification_MLflow.mlflow \
MLFLOW_TRACKING_USERNAME=SunilKumar-ugra \
MLFLOW_TRACKING_PASSWORD=ff039c8b90d721cb1c7f99b788c8bfe02f4d345b \
python script.py

export MLFLOW_TRACKING_URI=https://dagshub.com/SunilKumar-ugra/Kidney_Disease_Classification_MLflow.mlflow 

export MLFLOW_TRACKING_USERNAME=SunilKumar-ugra

export MLFLOW_TRACKING_PASSWORD=ff039c8b90d721cb1c7f99b788c8bfe02f4d345b

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/SunilKumar-ugra/Kidney_Disease_Classification_MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="SunilKumar-ugra"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ff039c8b90d721cb1c7f99b788c8bfe02f4d345b"

# Entity Folder
1. Goto entity folder  src-> wine_quality -> entity-> __init__.py 
2. Paste below code in __init__.py constructor file

In [7]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

into file 

In [8]:
%%writefile -a src/{project_name}/entity/__init__.py

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

Appending to src/cnnclf/entity/__init__.py


# Config folder 
1. Copy get_model_evaluation_config function 
2. Goto config folder src->wine_quality->config->configuration.py paste under ConfigurationManager class
3. Import ModelEvaluationConfig from wine_quality.entity using this line  

from wine_quality.entity import ModelEvaluationConfig:


In [10]:
project_name

'cnnclf'

In [11]:
from cnnclf.constants import *
from cnnclf.utils import read_yaml, create_directories,save_json

In [19]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/SunilKumar-ugra/Kidney_Disease_Classification_MLflow.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
%%writefile -a src/{project_name}/config/__init__.py
from Kidney_Disease_CnnClf.entity import EvaluationConfig
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/SunilKumar-ugra/Kidney_Disease_Classification_MLflow.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


Appending to src/cnnclf/config/__init__.py


# Component folder
1. Copy entire ModelEvaluation class
2. Goto Component folder [ src-> textSummarizer -> components - > model_evaluation.py ]
2. Paste ModelEvaluation class in model_evaluation.py
3. Import model_evaluation_config from entity using this line 

from wine_quality.entity import model_evaluation_config

In [27]:
project_name

'cnnclf'

In [24]:
#%%writefile -a src/{project_name}/components/model_evaluation_mlflow.py
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

Appending to src/cnnclf/components/model_evaluation_mlflow.py


In [25]:
# %%writefile -a src/{project_name}/components/model_evaluation_mlflow.py
# from cnnclf.entity import   EvaluationConfig

Appending to src/cnnclf/components/model_evaluation_mlflow.py


In [26]:
#%%writefile -a src/{project_name}/components/model_evaluation_mlflow.py

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

Appending to src/cnnclf/components/model_evaluation_mlflow.py


# Pipeline folder

1. Goto Component folder [ src-> textSummarizer -> pipeline - > stage_05_model_evaluation.py ]

2. Paste below code in stage_05_model_evaluation.py

In [12]:
''' 

from wine_quality import logger
from wine_quality.config. import ConfigurationManager
from wine_quality.components.model_evaluation import ModelEvaluation
class ModelTrainerTrainingPipeline:
    def __init__(self):
        pass
    def main(self):
'''

' \n\nfrom wine_quality import logger\nfrom wine_quality.config. import ConfigurationManager\nfrom wine_quality.components.model_evaluation import ModelEvaluation\nclass ModelTrainerTrainingPipeline:\n    def __init__(self):\n        pass\n    def main(self):\n'

3. Copy try block code 
4. Paste in main function in stage_05_model_evaluation.py

5. Import ConfigurationManager from config.configuration using this line  
from wine_quality.config.configuration import ConfigurationManager

6. Import ModelTrainer  from  components.model_trainer  using this line  
from wine_quality.components..model_evaluation  import class ModelEvaluation:


In [17]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [31]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e



[2023-10-28 12:51:05,004] - cnnclf - INFO - __init__ file - LineNum:31   - yaml file: config\config.yaml loaded successfully
[2023-10-28 12:51:05,011] - cnnclf - INFO - __init__ file - LineNum:31   - yaml file: params.yaml loaded successfully
[2023-10-28 12:51:05,013] - cnnclf - INFO - __init__ file - LineNum:50   - created directory at: artifacts


Found 139 images belonging to 2 classes.
9/9 [==============================] - 43s 5s/step - loss: 1.5603 - accuracy: 0.8273
[2023-10-28 12:51:48,858] - cnnclf - INFO - __init__ file - LineNum:64   - json file saved at: scores.json


2023/10/28 12:51:51 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-10-28 12:51:53,487] - tensorflow - INFO - builder_impl file - LineNum:804   - Assets written to: C:\Users\win10\AppData\Local\Temp\tmpbf1deedm\model\data\model\assets


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/10/28 12:53:04 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


In [ ]:
'''
Example
from wine_quality.config import ConfigurationManager
from wine_quality.components.model_evaluation import ModelEvaluation
from wine_quality import logger




class ModelEvaluationTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        model_evaluation_config = config.get_model_evaluation_config()
        model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
        model_evaluation_config.evaluate()
'''

In [28]:
project_name

'cnnclf'

In [29]:
%%writefile -a src/{project_name}/pipeline/stage_04_model_evaluation.py
from cnnclf import logger
from cnnclf.config import ConfigurationManager
from cnnclf.components.model_evaluation_mlflow import Evaluation

class EvaluationPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        eval_config = config.get_evaluation_config()
        evaluation = Evaluation(eval_config)
        evaluation.evaluation()
        evaluation.save_score()
        evaluation.log_into_mlflow()

Appending to src/cnnclf/pipeline/stage_04_model_evaluation.py


main file

In [30]:
%%writefile -a src/{project_name}/pipeline/stage_04_model_evaluation.py
STAGE_NAME = "Evaluation stage"
if __name__ == '__main__':
    try:
        logger.info(f"*******************")
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = EvaluationPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e

Appending to src/cnnclf/pipeline/stage_04_model_evaluation.py


In [ ]:
'''

1. Goto main.py and paste below code

STAGE_NAME = "Model Evaluation stage"
try: 
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   model_evaluation = ModelEvaluationTrainingPipeline()
   model_evaluation.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise e

2. Run main.py 
        
'''

In [ ]:
%%writefile -a main.py

STAGE_NAME = "Evaluation stage"
try:
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   model_evalution = EvaluationPipeline()
   model_evalution.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")

except Exception as e:
        logger.exception(e)
        raise e